<a href="https://colab.research.google.com/github/charantek1styearbtech/NLTK_ChatBOt/blob/main/chatbot_using_nltk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
lemmatizer=WordNetLemmatizer()

In [ ]:
stopwords=set(stopwords.words('english'))

Lemmatization is the process of converting words to their base or dictionary form

In [ ]:
def pre_processing(sentance):
    tokens=nltk.word_tokenize(sentance)
    tokens=[lemmatizer.lemmatize(word) for word in tokens if word not in stopwords]
    return tokens

feature extraction using sklearn.feature_extraction.text

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import json

In [ ]:
with open('intents.json') as file:
    dialogs=json.load(file)
dialogs

[{'tag': 'greeting',
  'patterns': ['Hi',
   'How are you?',
   'Is anyone there?',
   'Hello',
   'Good day',
   "What's up",
   'how are ya',
   'heyy',
   'whatsup',
   '??? ??? ??'],
  'responses': ['Hello!',
   'Good to see you again!',
   'Hi there, how can I help?'],
  'context_set': ''},
 {'tag': 'goodbye',
  'patterns': ['cya',
   'see you',
   'bye bye',
   'See you later',
   'Goodbye',
   'I am Leaving',
   'Bye',
   'Have a Good day',
   'talk to you later',
   'ttyl',
   'i got to go',
   'gtg'],
  'responses': ['Sad to see you go :(',
   'Talk to you later',
   'Goodbye!',
   'Come back soon'],
  'context_set': ''},
 {'tag': 'creator',
  'patterns': ['what is the name of your developers',
   'what is the name of your creators',
   'what is the name of the developers',
   'what is the name of the creators',
   'who created you',
   'your developers',
   'your creators',
   'who are your developers',
   'developers',
   'you are made by',
   'you are made by whom',
   'who

In [ ]:
sentences=[]
labels=[]
label_map={}
for intent in dialogs:
  for pattern in intent['patterns']:
        sentences.append(pattern)
        labels.append(intent['tag'])
        label_map[intent['tag']] = len(label_map)

In [ ]:
tfid=TfidfVectorizer(tokenizer=pre_processing)

In [ ]:
x=tfid.fit_transform(sentences)
y=[label_map[i] for i in labels]

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
from sklearn.preprocessing import LabelBinarizer

In [ ]:
encode=LabelBinarizer()
y=encode.fit_transform(y)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [ ]:
num_classes=len(label_map)

In [ ]:
chatbot=Sequential()

In [ ]:
chatbot.add(Dense(128,input_shape=(x.shape[1],),activation='relu'))
chatbot.add(Dropout(0.5))
chatbot.add(Dense(64,activation='relu'))
chatbot.add(Dropout(0.5))
chatbot.add(Dense(num_classes,activation='softmax'))

In [ ]:
chatbot.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
chatbot.fit(x.toarray(),y,epochs=200,batch_size=5,verbose=1)

Epoch 1/200
81/81 [==============================] - 1s 2ms/step - loss: 3.6203 - accuracy: 0.0519
Epoch 2/200
81/81 [==============================] - 0s 2ms/step - loss: 3.5195 - accuracy: 0.0889
Epoch 3/200
81/81 [==============================] - 0s 2ms/step - loss: 3.4244 - accuracy: 0.1086
Epoch 4/200
81/81 [==============================] - 0s 2ms/step - loss: 3.3180 - accuracy: 0.1284
Epoch 5/200
81/81 [==============================] - 0s 2ms/step - loss: 3.2162 - accuracy: 0.1679
Epoch 6/200
81/81 [==============================] - 0s 2ms/step - loss: 3.0578 - accuracy: 0.2000
Epoch 7/200
81/81 [==============================] - 0s 2ms/step - loss: 2.9385 - accuracy: 0.2667
Epoch 8/200
81/81 [==============================] - 0s 2ms/step - loss: 2.8505 - accuracy: 0.2667
Epoch 9/200
81/81 [==============================] - 0s 2ms/step - loss: 2.7092 - accuracy: 0.3086
Epoch 10/200
81/81 [==============================] - 0s 4ms/step - loss: 2.5890 - accuracy: 0.3210
Epoch 11/

In [ ]:
import numpy as np
test_vectorized = tfid.transform([input("You:")])
indices = test_vectorized.nonzero()
values = test_vectorized.data
dense_shape = test_vectorized.shape

# Ensure indices are in the correct shape
indices = tuple(zip(indices[0], indices[1]))

# Create SparseTensor
test_sparse_tensor = tf.sparse.SparseTensor(indices, values, dense_shape)

# Reorder SparseTensor with sorted indices
test_sparse_tensor_reordered = tf.sparse.reorder(test_sparse_tensor)
prediction = chatbot.predict(test_sparse_tensor_reordered)
intent_index = np.argmax(prediction)
intent_label = encode.classes_[intent_index]
print(intent_label)

You:contact info
1/1 [==============================] - 0s 8ms/step
6


In [ ]:
import random
output_messages=dialogs[intent_label-1]['responses']
print(random.choice(output_messages))

You can contact at: +918688197832
